In [1]:
import pandas
from decimal import Decimal
from datetime import datetime
from dateutil import relativedelta
from collections import namedtuple
import operator
import functools
import metrics

In [2]:
def fmt_money(number):
    return "${:,.0f}".format(number)

In [3]:
Monthly = namedtuple('Monthly', ['date', 'stocks', 'dividend', 'inflation', 'margin_rate'])

In [4]:
class Rate:
    def __init__(self, initial_value):
        self.value = initial_value
        
    def next(self, new_value):
        delta = (new_value / self.value) - 1
        self.value = new_value
        return delta
    
class Margin_Rates:
    def __init__(self, add=0.015):
        self.data = pandas.read_csv('FEDFUNDS.csv', index_col=0, parse_dates=True)
        self.add = add
    
    def get_margin(self, date):
        row = self.data.loc[date]
        rate = row['FEDFUNDS']
        rate /= 100
        rate += self.add
        return rate

class US_1871_Monthly:
    def __init__(self):
        self.data = pandas.read_csv('US_1871_Monthly.csv', index_col=0, parse_dates=True)
        self.margin_rates = Margin_Rates()
        
    def iter_from(self, year, month, length=None):
        self.stocks_price = Rate(0.0)
        self.inflation = Rate(0.0)

        count = 0
        d = datetime(year, month, 1)
        for row in self.data.loc[d:].iterrows():
            yield self.fmt(row[1])
            count += 1
            if length and count >= length:
                raise StopIteration

    def fmt(self, row):
        inflation = self.inflation.next(row['CPI'])
        stocks = self.stocks_price.next(row['S&P 500 Price'])
        dividend = (row['S&P 500 Dividend'] / row['S&P 500 Price']) / 12
        date = row.name
        margin = self.margin_rates.get_margin(date)
        return Monthly(date=date, stocks=stocks, dividend=dividend, inflation=inflation, margin_rate=margin)

In [5]:
class Simulation:
    def __init__(self, year, month, length=30):
        self.returns = US_1871_Monthly()

        # we need to start at the previous month so that "change since last month"
        # numbers are calculated correctly. That also means the simulation needs to
        # run for one extra month
        start = (year * 12) + month - 1
        previous_month = start - 1
        (prev_year, prev_month) = (int(previous_month / 12), (previous_month % 12)+1)
        r = self.returns.iter_from(prev_year, prev_month, length=(length*12)+1)
        next(r)
        
        self.data = r
        
        self.portfolio = 1000000
        self.withdrawal = self.portfolio / 25 / 12 # 4% rule
        self.real_starting_portfolio = self.portfolio
        self.withdraw_margin = False
        self.debt = 0

        self.log = pandas.DataFrame(columns=['withdrawal',
                                             'portfolio', 
                                             'debt', 
                                             'real_start_port', 
                                             'used_margin', 
                                             'margin_call', 
                                             'debt_paydown', 
                                             'dividend'])

    def do_strategy(self, monthly):
        pass

    def run(self):
        for monthly in self.data:
            # each month we...
            # withdraw money for the beginning of the month.
            ## adjust withdrawal for inflation
            self.withdrawal *= (1 + monthly.inflation)
            self.real_starting_portfolio *= (1 + monthly.inflation)
            
            extra_logs = self.do_strategy(monthly)

            # and then adjust our debt & portfolio by whatever
            self.portfolio *= (1 + monthly.stocks)
            self.debt *= (1 + (monthly.margin_rate/12))
            
            self.log.loc[monthly.date] = (extra_logs[0],
                                          self.portfolio,
                                          self.debt,
                                          self.real_starting_portfolio,
                                          self.withdraw_margin,
                                          extra_logs[1],
                                          extra_logs[2],
                                          extra_logs[3])

In [6]:
def p_loc(df, loc):
    row = df.iloc[loc]
    for c in df.columns:
        print(c, fmt_money(row[c]))

In [7]:
class Strategy_2(Simulation):
    USE_TOTAL_RETURNS = False
    USE_MARGIN = True
    def do_strategy(self, monthly):
        # check if we hit the lower guardrail and need to start using debt
        if self.portfolio <= self.real_starting_portfolio * .7:
            self.withdraw_margin = True

        # check if we've recovered past the upper guardrail and should stop using debt
        if self.portfolio >= self.real_starting_portfolio * .8:
            self.withdraw_margin = False

        margin_call = 0
        # check if we are hitting margin calls and can't use debt
        if self.debt > (self.portfolio / 2):
            self.withdraw_margin = False
            # what's more we have to fulfill the margin call and sell
            # when we sell, we actually increase the amount we need to sell
            # because the selling drops up further below the margin call limit
            margin_call = (2 * self.debt) - self.portfolio
            margin_call = min(self.portfolio, margin_call)
            self.portfolio -= margin_call
            self.debt -= margin_call
            
        # first we use dividends for our expenses
        withdrawal = self.withdrawal
        withdrawal = actual_withdrawal = min(withdrawal, self.portfolio)
        dividend = self.portfolio * monthly.dividend

        # this is the traditional "total return" kind of model, where
        # dividends just silently increase the portfolio
        if self.USE_TOTAL_RETURNS:
            self.portfolio += dividend
        else:
            # but this uses a more realistic "spend your dividends" model
            # it results in *dramatically* different outcomes
            if dividend < withdrawal:
                withdrawal -= dividend
                # the rest will have to come from margin or the portfolio
            else:
                # the dividend is more than our withdrawal, so the rest of it
                # gets added to the portfolio
                dividend -= withdrawal
                self.portfolio += dividend
                withdrawal = 0

        if self.USE_MARGIN and self.withdraw_margin:
            self.debt += withdrawal
        else:
            self.portfolio -= withdrawal

        assert self.portfolio >= 0, "wd: %d, p: %d" % (withdrawal, self.portfolio)
            
        DEBT_PAYOFF_THRESHOLD = 1
            
        # check if, after the withdrawal, if we have enough "extra" to pay down debt
        excess = 0
        if self.debt > 0 and self.portfolio >= self.real_starting_portfolio * DEBT_PAYOFF_THRESHOLD:
            excess = self.portfolio - (self.real_starting_portfolio * DEBT_PAYOFF_THRESHOLD)
            excess = min(self.debt, excess)
            self.debt -= excess
            self.portfolio -= excess

        return (actual_withdrawal, margin_call, excess, dividend)



In [9]:
s2 = Strategy_2(1968, 1, length=30)
s2.USE_MARGIN = False
s2.run()
s2.log.tail()
#last = s2.log.iloc[-1]
#print(last.portfolio / last.real_start_port)
#print(last.portfolio / (last.withdrawal * 12))
s2.log.to_csv('margin_guardrails.csv')

/Users/justus/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  
/Users/justus/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:35: DeprecationWarning: generator 'US_1871_Monthly.iter_from' raised StopIteration


In [59]:
def get_max_debt(log):
#    return max([x.debt for (r, x) in log.iterrows()])    
    return max([x.debt / (x.portfolio or 1) for (r, x) in log.iterrows()])

In [60]:
def used_margin(log):
    return functools.reduce(operator.or_, [x[1].used_margin for x in log.iterrows()])

In [61]:
#s2.log.to_csv('margin_guardrails.csv')

In [62]:
comp = pandas.DataFrame(columns=['portfolio_no_margin',
                                 'portfolio_margin',
                                 'debt',
                                 'used_margin',
                                 'networth'])

# 1955 is the furthest back our Fed Funds rate data goes
LENGTH = 30
for year in range(1955, 2017-LENGTH+1):
#    print(year)
    s = Strategy_2(year, 1, length=LENGTH)
    s.run()
    margin = s.log.iloc[-1].portfolio
    debt = s.log.iloc[-1].debt
    used_m = used_margin(s.log)
    print(year, ",", get_max_debt(s.log))
    
    s = Strategy_2(year, 1, length=30)
    s.USE_MARGIN = False
    s.run()
    no_margin = s.log.iloc[-1].portfolio

    comp.loc[year] = (no_margin, margin, debt, used_m, margin-debt)

/Users/justus/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  


1955 , 0.0
1956 , 0.0
1957 , 0.0
1958 , 0.0
1959 , 0.01032529993492957
1960 , 0.09153669638301133
1961 , 0.017410555081294042
1962 , 0.34017388620973615
1963 , 0.09583978125787486
1964 , 0.3073056783655628
1965 , 0.5287681354397474
1966 , 0.572392212430798
1967 , 0.32549473611333396
1968 , 0.5294007275592785
1969 , 1551.3543128341707
1970 , 0.2797377404745856
1971 , 0.20433612815882815
1972 , 0.2614363060833802
1973 , 0.4269098463414265
1974 , 0.08267294490113836
1975 , 0.0
1976 , 0.0
1977 , 0.0023115807892225776
1978 , 0.0
1979 , 0.0
1980 , 0.0
1981 , 0.0
1982 , 0.0
1983 , 0.0
1984 , 0.0
1985 , 0.0
1986 , 0.0
1987 , 0.0


In [20]:
comp.to_csv('margin_guardrails.csv')